In [49]:
import gurobipy as gp
from gurobipy import GRB
import numpy as np
import pandas as pd

In [37]:
# Load the data
cost_matrix_df = pd.read_csv('/Users/mahinbindra/Downloads/cost_matrix.csv')
distributions_df = pd.read_csv('/Users/mahinbindra/Downloads/distributions.csv')

In [38]:
# Convert cost matrix to a NumPy array for easier access
cost_matrix = cost_matrix_df.to_numpy()

In [50]:
# Constants
ordering_cost = 24.44
num_scenarios = 1000  # Number of Monte Carlo simulations
np.random.seed(0)  # For reproducibility

In [51]:
# Generate demand scenarios using Monte Carlo simulation
demand_scenarios = np.array([
    np.random.binomial(n=row['n'], p=row['p'], size=num_scenarios)
    for index, row in distributions_df.iterrows()
])

In [52]:
# Initialize Gurobi model
model = gp.Model("Two_Stage_Stochastic_Baffin_Bay_Retail_Network")

In [53]:
# First stage decision variables: initial order quantities
y = model.addVars(len(distributions_df), lb=0, vtype=GRB.CONTINUOUS, name="order_qty")

In [54]:
# Second stage decision variables: transshipment quantities for each scenario
x = model.addVars(len(distributions_df), len(distributions_df), num_scenarios, lb=0, vtype=GRB.CONTINUOUS, name="transshipment")

In [55]:
# Objective function: Minimize total cost
total_cost = ordering_cost * gp.quicksum(y[i] for i in range(len(distributions_df))) + \
             gp.quicksum(cost_matrix[i, j] * x[i, j, k] for i in range(len(distributions_df)) 
                         for j in range(len(distributions_df)) if i != j for k in range(num_scenarios))
model.setObjective(total_cost, GRB.MINIMIZE)

In [57]:
# Constraints
for k in range(num_scenarios):
    for i in range(len(distributions_df)):
        # Ensure demand is met in each scenario
        model.addConstr(
            y[i] + gp.quicksum(x[j, i, k] for j in range(len(distributions_df)) if j != i) >= demand_scenarios[i, k],
            name=f"demand_met_{i}_{k}"
        )
        # Transshipment limits based on ordered quantities and scenario demands
        for j in range(len(distributions_df)):
            if i != j:
                model.addConstr(
                    x[i, j, k] <= y[i] - demand_scenarios[i, k],
                    name=f"transship_limit_{i}_{j}_{k}"
                )

In [58]:
# Solve the model
model.optimize()

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (mac64[arm] - Darwin 23.0.0 23A344)

CPU model: Apple M1
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 450000 rows, 225015 columns and 1290000 nonzeros
Model fingerprint: 0x858fbcbc
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 2e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+01, 1e+02]
Presolve removed 450000 rows and 225015 columns
Presolve time: 0.20s
Presolve: All rows and columns removed
Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.8101960e+04   0.000000e+00   0.000000e+00      0s

Solved in 0 iterations and 0.30 seconds (0.45 work units)
Optimal objective  3.810196000e+04


In [59]:
# Output results
if model.status == GRB.OPTIMAL:
    print("Optimal value (total cost): $", model.ObjVal)
    print("Order quantities (y_i):")
    for i in range(len(distributions_df)):
        print(f"Retailer {i+1}: {y[i].X:.2f} units")
else:
    print("No optimal solution found.")

Optimal value (total cost): $ 38101.96000000001
Order quantities (y_i):
Retailer 1: 104.00 units
Retailer 2: 68.00 units
Retailer 3: 112.00 units
Retailer 4: 84.00 units
Retailer 5: 68.00 units
Retailer 6: 77.00 units
Retailer 7: 126.00 units
Retailer 8: 119.00 units
Retailer 9: 102.00 units
Retailer 10: 109.00 units
Retailer 11: 94.00 units
Retailer 12: 110.00 units
Retailer 13: 121.00 units
Retailer 14: 133.00 units
Retailer 15: 132.00 units
